In [22]:
import requests
from bs4 import BeautifulSoup
import csv

def extraer_transacciones_openinsider(simbolo_ticker):
    """
    Extrae transacciones de acciones de OpenInsider y las guarda en un archivo CSV.

    Args:
        simbolo_ticker (str): El símbolo ticker de la acción (por ejemplo, 'META').
    """

    url = f"http://openinsider.com/screener?s={simbolo_ticker}&o=&pl=&ph=&ll=&lh=&fd=0&fdr=&td=0&tdr=&fdlyl=&fdlyh=&daysago=&xp=1&xs=1&vl=&vh=&ocl=&och=&sic1=-1&sicl=100&sich=9999&grp=0&nfl=&nfh=&nil=&nih=&nol=&noh=&v2l=&v2h=&oc2l=&oc2h=&sortcol=2&cnt=1000&page=1"
    
    transacciones = []
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lanza una excepción para códigos de error HTTP (4xx o 5xx)

        soup = BeautifulSoup(response.content, 'html.parser')
        tabla = soup.find('table', {'class': 'tinytable'})

        if tabla:
            encabezados = [th.text.strip() for th in tabla.find_all('th')]
            filas = tabla.find_all('tr')[1:]  # Ignora la fila de encabezado
            for fila in filas:
                celdas = fila.find_all('td')
                if len(celdas) >= len(encabezados):  # Asegura que haya suficientes celdas
                    transaccion = {encabezados[i]: celdas[i].text.strip() for i in range(len(encabezados))}
                    transacciones.append(transaccion)
        else:
            print("No se encontró la tabla de transacciones.")

    except requests.exceptions.RequestException as e:
        print(f"Error al obtener la página: {e}")

    if transacciones:
        guardar_transacciones_csv(transacciones, f"{simbolo_ticker}_transaccionesOPENINSIDER.csv")
    else:
        print("No se encontraron transacciones.")

def guardar_transacciones_csv(transacciones, nombre_archivo):
    """
    Guarda una lista de diccionarios de transacciones en un archivo CSV.

    Args:
        transacciones (list): Lista de diccionarios de transacciones.
        nombre_archivo (str): El nombre del archivo CSV.
    """

    with open(nombre_archivo, 'w', newline='', encoding='utf-8') as archivo_csv:
        nombres_campos = transacciones[0].keys()
        escritor_csv = csv.DictWriter(archivo_csv, fieldnames=nombres_campos)

        escritor_csv.writeheader()
        for transaccion in transacciones:
            escritor_csv.writerow(transaccion)

    print(f"Transacciones guardadas en {nombre_archivo}")

if __name__ == "__main__":
    simbolo_ticker = "META"  # actualiza el ticker de la acción, pilas con las mayúsculas
    extraer_transacciones_openinsider(simbolo_ticker)


Transacciones guardadas en META_transaccionesOPENINSIDER.csv


In [ ]:
%pip install yfinance # NO ES API OFICIAL DE YAHOO FINANCE, SE OBTUVO DE GITHUB


     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 3.0/3.0 MB 43.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for peewee: filename=peewee-3.17.9-py3-none-any.whl size=139127 sha256=045ad303e55fef3c79b0e083e9cbc7f51bb25272d9a8e650805bbe1818870ab4
  Stored in directory: c:\users\santi\appdata\local\pip\cache\wheels\43\ef\2d\2c51d496bf084945ffdf838b4cc8767b8ba1cc20eb41588831
Successfully built peewee


In [28]:
import yfinance as yf
import pandas as pd

def obtener_precios_acciones(tickers, fecha_inicio="2015-01-01"):
    """
    Obtiene los precios de las acciones de una lista de símbolos ticker y los guarda en un archivo CSV.

    Args:
        tickers (list): Lista de símbolos ticker de las acciones (por ejemplo, ["AAPL", "AMZN", "MSFT", "META", "GOOG"]).
        fecha_inicio (str): La fecha de inicio para los datos (en formato YYYY-MM-DD).

    Returns:
        pandas.DataFrame: Un DataFrame con los precios de las acciones, o None si hay un error.
    """

    try:
        data = yf.download(tickers, start=fecha_inicio)
        return data
    except Exception as e:
        print(f"Error al obtener los datos: {e}")
        return None

if __name__ == "__main__":
    simbolos_ticker = ["AAPL"]
    datos_acciones = obtener_precios_acciones(simbolos_ticker)

    if datos_acciones is not None:
        print(datos_acciones)

        # Guardar los datos en un archivo CSV
        datos_acciones.to_csv(f"{simbolos_ticker[0]}_precios_acciones.csv")
        print("Datos guardados en precios_acciones.csv")
        


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2015-01-02   24.320431   24.789800   23.879980   24.778677  212818400
2015-01-05   23.635284   24.169164   23.448427   24.089082  257142000
2015-01-06   23.637516   23.897782   23.274922   23.699802  263188400
2015-01-07   23.968958   24.069060   23.735385   23.846610  160423600
2015-01-08   24.889898   24.947736   24.180283   24.298183  237458000
...                ...         ...         ...         ...        ...
2025-03-21  218.270004  218.839996  211.279999  211.559998   94127800
2025-03-24  220.729996  221.479996  218.580002  221.000000   44299500
2025-03-25  223.750000  224.100006  220.080002  220.770004   34493600
2025-03-26  221.529999  225.020004  220.470001  223.509995   34532700
2025-03-27  223.850006  224.990005  220.559998  221.389999   37049500

[2574 rows x 5 colu